In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.CDSSMPreprocessor()
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 2118/2118 [00:00<00:00, 6751.07it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 18841/18841 [00:05<00:00, 3730.90it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 1614998/1614998 [00:00<00:00, 3407400.32it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8756.29it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4251.64it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 59226.17it/s]
Processing text_left with chain_transform of NgramLetterUnit => WordHashingUnit: 100

In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 8118.95it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 4421.77it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 63304.89it/s]
Processing text_left with chain_transform of NgramLetterUnit => WordHashingUnit: 100%|██████████| 122/122 [00:00<00:00, 6042.02it/s]
Processing text_right with chain_transform of NgramLetterUnit => WordHashingUnit: 100%|██████████| 1115/1115 [00:00<00:00, 2165.76it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 4295.23it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 2300/

In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [21]:
model = mz.models.CDSSMModel()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['filters'] = 300
model.params['kernel_size'] = 3
model.params['strides'] = 1
model.params['padding'] = 'same'
model.params['conv_activation_func'] = 'tanh'
model.params['w_initializer'] = 'glorot_normal'
model.params['b_initializer'] = 'zeros'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 300
model.params['mlp_num_fan_out'] = 128
model.params['mlp_activation_func'] = 'tanh'
model.params['dropout_rate'] = 0.8
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to CDSSMModel.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10, 9644)     0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 40, 9644)     0                                            
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 10, 300)      8679900     text_left[0][0]                  
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 40, 300)      8679900     text_right[0][0]                 
_________________________________________________________________________

In [22]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=2, num_neg=1, batch_size=64, shuffle=True)

In [23]:
history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=1, use_multiprocessing=False)

Epoch 1/20
32/32 [==============================] - 24s 748ms/step - loss: 0.7844
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.483600 - normalized_discounted_cumulative_gain@5(0):0.553078 - mean_average_precision(0):0.512808
Epoch 2/20
32/32 [==============================] - 29s 898ms/step - loss: 0.5235
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.488520 - normalized_discounted_cumulative_gain@5(0):0.561723 - mean_average_precision(0):0.510881
Epoch 3/20
32/32 [==============================] - 23s 706ms/step - loss: 0.2984
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.484800 - normalized_discounted_cumulative_gain@5(0):0.555596 - mean_average_precision(0):0.512422
Epoch 4/20
32/32 [==============================] - 25s 775ms/step - loss: 0.1954
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.475072 - normalized_discounted_cumulative_gain@5(0):0.562940 - mean_average_precision(0):0.512383
Epoch 5/